In [1]:
import undetected_chromedriver as uc
import re
from datetime import datetime
from lxml import html

In [2]:
from scraping_tools import get_html, scrape_property
from parsing_tools import extract_thumbs

In [3]:
urls = ['https://www.aruodas.lt/butai-vilniuje-snipiskese-kalvariju-g-foto-yra-vizualizacija-parduodamas-geras-1-3308709/',
        'https://www.aruodas.lt/butai-vilniuje-snipiskese-apkasu-g-parduodamas-k-butas-kv-a-yra-1-3295912/',
'https://www.aruodas.lt/butai-vilniuje-fabijoniskese-s-staneviciaus-g-parduodamas-jaukus-suremontuotas-k-butas-1-3297050/',
'https://www.aruodas.lt/butai-vilniuje-senamiestyje-saltiniu-g-parduodamos-negyvenamos-patalpos-buvusi-1-3306541/',
'https://www.aruodas.lt/butai-trakuose-lentvario-m-fabriko-g-parduodamas-izoliuotu-kambariu-butas-su-1-3253535/?utm_source=aruodas&utm_medium=ad&utm_campaign=vip-other',
]

properties = []

for url in urls:
    properties.append(scrape_property(url))
